In [1]:
# Install necessary packages
!pip uninstall llama-index -y
!pip install -U llama-index
!pip install -U gradio
!pip install -U transformers bitsandbytes
!pip install llama-index-embeddings-langchain
!pip install -U langchain-community
!pip install llama-index qdrant_client torch transformers
!pip install llama-index-llms-ollama
!pip install -U langchain-huggingface


Found existing installation: llama-index 0.11.15
Uninstalling llama-index-0.11.15:
  Successfully uninstalled llama-index-0.11.15
  Using cached llama_index-0.11.15-py3-none-any.whl.metadata (11 kB)
Using cached llama_index-0.11.15-py3-none-any.whl (6.8 kB)


In [2]:
from huggingface_hub import login

login("YOUR_TOKEN_HERE")

HF_HUB_ENABLE_HF_TRANSFER=1

# prompt: create code for clearing torch ram

import torch
torch.cuda.empty_cache()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
!pip install llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 5.0 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoProcessor
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import torch
import gradio as gr
import json

from transformers import MllamaForConditionalGeneration, AutoProcessor
import torch

# Load the model and processor
model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # Use bfloat16 for better memory handling
    device_map="auto",  # Automatically map to available devices (like GPU)
)
processor = AutoProcessor.from_pretrained(model_id)


# Load product data
with open("/content/combined_products_(1).json", "r", encoding="utf-8") as file:
    product_data = json.load(file)

# Create Documents for LlamaIndex
documents = []
for product in product_data:
    product_desc = product.get("description", "")
    product_title = product.get("title", "")
    product_attrs = product.get("attributes", {})
    product_text = f"Title: {product_title}\nDescription: {product_desc}\nAttributes: {json.dumps(product_attrs)}"
    documents.append(Document(text=product_text))



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
# Initialize embedding model and create the index
local_embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [9]:

index = VectorStoreIndex.from_documents(documents, embed_model=local_embedding_model)


In [10]:
index.storage_context.persist("rag_index")

In [6]:
from llama_index.core import StorageContext, load_index_from_storage#,ALI;IYO
storage_context = StorageContext.from_defaults(persist_dir="rag_index")

new_index = load_index_from_storage(
    storage_context,
    embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5") # explicitly set the embedding model used during index creation
)


In [ ]:
# Import necessary libraries ÇALIŞITO
import gradio as gr
from PIL import Image
import json

import torch
import re

# Utility function to clean and preprocess RAG data
def clean_text(text):
    # Remove any inf, nan, or invalid values
    if isinstance(text, float):
        if torch.isnan(torch.tensor(text)) or torch.isinf(torch.tensor(text)):
            return ""
    if isinstance(text, str):
        # Replace any unexpected characters (e.g., non-ASCII)
        return re.sub(r'[^\x00-\x7F]+', ' ', text)
    return str(text)

def preprocess_rag_data(rag_data):
    clean_data = []
    for item in rag_data:
        # Extract text and metadata properly from NodeWithScore
        node = item.node  # Get the actual node from NodeWithScore
        product_title = clean_text(node.extra_info.get("title", "N/A")) if hasattr(node, 'extra_info') else "N/A"
        product_description = clean_text(node.extra_info.get("description", "N/A")) if hasattr(node, 'extra_info') else "N/A"
        product_features = clean_text(json.dumps(node.extra_info.get("attributes", {}))) if hasattr(node, 'extra_info') else "N/A"

        cleaned_item = {
            "title": product_title,
            "description": product_description,
            "attributes": product_features
        }
        clean_data.append(cleaned_item)
    return clean_data


def query_product_information(query):
    # Use the retriever to get relevant documents
    retriever = new_index.as_retriever()
    retrieved_docs = retriever.retrieve(query)

    # Clean and preprocess the RAG data
    clean_docs = preprocess_rag_data(retrieved_docs[:10])  # Top 3 results

    product_infos = []
    for doc in clean_docs:
        product_title = doc.get("title", "N/A")
        product_description = doc.get("description", "N/A")
        product_features = doc.get("attributes", "N/A")

        structured_product_info = f"""
        Product Title: {product_title}
        Product Description: {product_description}
        Product Features: {product_features}
        """
        product_infos.append(structured_product_info)

    # Join all product info together for final response
    response = "\n\n".join(product_infos)
    return response


def process_input(input_text):
    """
    Function to process input text, extracting relevant content from tagged sections.
    """
    header_start = "<|start_header_id|>"
    header_end = "<|end_header_id|>"
    eot_id = "<|eot_id|>"

    header_start_idx = input_text.rfind(header_start) + len(header_start)
    header_end_idx = input_text.rfind(header_end)
    bot_name = input_text[header_start_idx:header_end_idx].strip()

    message_start_idx = header_end_idx + len(header_end)
    if input_text.endswith(eot_id):
        message_end_idx = input_text.rfind(eot_id)
    else:
        message_end_idx = len(input_text)

    message = input_text[message_start_idx:message_end_idx].strip()

    return f"{message}"

def create_response(uploaded_image, content, max_new_tokens=400, process_input_boolean=True):
    # Process the text input and ensure the image is included
    input_text = processor.apply_chat_template(content, add_generation_prompt=True)
    # Include image if provided
    inputs = processor(images=uploaded_image, text=input_text, return_tensors="pt").to(model.device)

    # Clean up any bad inputs before generating
    for key, value in inputs.items():
        if torch.isnan(value).any() or torch.isinf(value).any():
            raise ValueError(f"Invalid input detected in {key}: contains NaN or inf values")

    # Generate the output based on text and image inputs
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)

    # Decode and process the result
    bot_message = processor.decode(output_ids[0])
    if process_input_boolean:
        bot_message = process_input(bot_message)

    return bot_message.strip()



# Now the RAG-based product information is integrated into get_product_name
def get_product_name(explanation, uploaded_image):
    """
    Generates product name using explanation, image, and retrieved RAG data.
    """
    # RAG retrieval happens here
    rag_response = query_product_information(explanation)

    # Combine RAG response and explanation
    message = f"""Based on the provided product description and image: '{explanation}'
                  and look at the retrieved information to observe how product names and descriptions are formatted and written do not take any features from them just analyze: '{rag_response}', explain the product at the image.Most importantly Do not add any other information that is not given to you."""

    content = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": message}]}]
    name_answer = create_response(uploaded_image, content, max_new_tokens=100)

    content.append({"role": "assistant", "content": [{"type": "text", "text": name_answer}]})
    message_2 = "What is the name of the product?"
    content.append({"role": "user", "content": [{"type": "text", "text": message_2}]})

    name_answer_2 = create_response(uploaded_image, content, max_new_tokens=100)
    content.append({"role": "assistant", "content": [{"type": "text", "text": name_answer_2}]})

    message_3 = "Only provide the name of the product."
    content.append({"role": "user", "content": [{"type": "text", "text": message_3}]})

    name_answer_3 = create_response(uploaded_image, content, max_new_tokens=20)
    return name_answer_3

# Here, RAG integration continues for the target audience
def get_target_audience(product_name, uploaded_image):
    """
    Uses the product name and image to generate the ideal target audience.
    """
    rag_response = query_product_information(product_name)

    message = f"""Based on {product_name}, what is the ideal target audience for this product?"""

    content = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": message}]}]
    target_answer = create_response(uploaded_image, content)

    content.append({"role": "assistant", "content": [{"type": "text", "text": target_answer}]})
    message_2 = f"""Which type of people can be our target audience and why?"""
    content.append({"role": "user", "content": [{"type": "text", "text": message_2}]})

    target_answer_2 = create_response(uploaded_image, content)
    content.append({"role": "assistant", "content": [{"type": "text", "text": target_answer_2}]})

    message_3 = f"""I want one target audience that includes all of these."""
    content.append({"role": "user", "content": [{"type": "text", "text": message_3}]})
    target_answer_3 = create_response(uploaded_image, content, max_new_tokens=50)

    message_4 = f"""Only provide name of the audience."""
    content.append({"role": "user", "content": [{"type": "text", "text": message_4}]})
    target_answer_4 = create_response(uploaded_image, content)

    return target_answer_4

def get_all_features(product_name, uploaded_image):
    """
    Retrieves detailed features of the product using both the image and product name.
    """
    rag_response = query_product_information(product_name)

    message = f"""Please provide details about features of {product_name} and look at how some items features to see as an exampledo not use any dta from this just analyze '{rag_response}' at the image. Most importantly Do not add any other information that is not given to you."""
    content = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": message}]}]
    f_answer = create_response(uploaded_image, content)

    content.append({"role": "assistant", "content": [{"type": "text", "text": f_answer}]})
    message_2 = f"""What are the product's features based on the image and provided information? Most importantly Do not add any other information that is not given to you."""
    content.append({"role": "user", "content": [{"type": "text", "text": message_2}]})

    f_answer_2 = create_response(uploaded_image, content)
    content.append({"role": "assistant", "content": [{"type": "text", "text": f_answer_2}]})

    message_3 = """Talk about features the item can have but no specifics. Only provide the format. Most importantly Do not add any other information that is not given to you."""
    content.append({"role": "user", "content": [{"type": "text", "text": message_3}]})

    f_answer_3 = create_response(uploaded_image, content)
    all_features = f_answer_3

    return all_features

def get_strong_message(product_name, uploaded_image, target_audience, all_features):
    """
    Creates a comprehensive e-commerce description based on product name, image, audience, and features.
    """
    rag_response = query_product_information(product_name)

    message = f"""Most importantly Do not add any other information that is not given to you.Please provide a comprehensive but short and meaningful overview of {product_name}, based on the image, look at the retrieved information to observe how product names and descriptions are formatted and written do not take any features from them just analyze: '{rag_response}', and the features: {all_features}."""
    content = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": message}]}]
    sm_answer = create_response(uploaded_image, content)

    content.append({"role": "assistant", "content": [{"type": "text", "text": sm_answer}]})
    message_2 = f"""Assume you are a {target_audience}, which features would you want from this product?Do not add any other information that is not given to you."""
    content.append({"role": "user", "content": [{"type": "text", "text": message_2}]})

    sm_answer_2 = create_response(uploaded_image, content)
    content.append({"role": "assistant", "content": [{"type": "text", "text": sm_answer_2}]})

    message_3 = f"""Create an e-commerce name and after that explanation using these features {all_features} and the previous information. Most importantly Do not add any other information that is not given to you. Also write alignments natural not like sections add evrything together in a paragraph. Don't use too much advertising slogans. the output should just be the suggested poduct name and right after description.dont make comments on the background"""
    content.append({"role": "user", "content": [{"type": "text", "text": message_3}]})

    sm_answer_3 = create_response(uploaded_image, content)
    return sm_answer_3

# Full description generation with RAG included
def description_generate(message, uploaded_image):
    """
    Full description generation using RAG, product name, target audience, and features.
    """
    product_name = get_product_name(message, uploaded_image)
    target_audience = get_target_audience(product_name, uploaded_image)
    all_features = get_all_features(product_name, uploaded_image)
    strong_message = get_strong_message(product_name, uploaded_image, target_audience, all_features)

    return strong_message

# Gradio interface setup
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=2):
            img_input = gr.Image(type="pil", label="Upload Product Image")
            msg = gr.Textbox(placeholder="Enter a short product description.", label="Product Description")
            with gr.Row():
                with gr.Column():
                    chat_button = gr.Button("Generate Description")
                with gr.Column():
                    clear = gr.Button("Clear 🗑️")
        with gr.Column(scale=4):
            result_box = gr.Textbox(label="Generated Product Description", interactive=False, show_copy_button=True)

    chat_button.click(description_generate, [msg, img_input], result_box)
    clear.click(lambda: (None, "", ""), outputs=[img_input, msg, result_box])

# Launch Gradio Interface
demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://b6bff1910f847b0520.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
